In [ ]:
# ----------------------------------------------------------------------------
#----------------------------- BEGIN CV DATA  -----------------------------
# ----------------------------------------------------------------------------

# !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!
#########>>>>#########>>>>#########>>>>#########>>>>#########>>>>#########>>>>
# ------------- BEFORE RUNNING THIS CODE - YOU NEED TO SAVE -------------
# -------- THE CVS.zip in Google Drive-main page. --------
#########>>>>#########>>>>#########>>>>#########>>>>#########>>>>#########>>>>
# !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!  !!!


# The code:
#  [1] Creates in the google drive a couple of folders and files:
#       [i] Main Folder named CVS
#           "../gdrive/My Drive/CVS"
#           Contains all folthers and .csv files as answers
#      [ii] CVs - "../gdrive/My Drive/CVS/CVs"
#           Contains all the CVs.docx
#     [iii] Results - "../gdrive/My Drive/CVS/Results"
#           Contains new colored CVs.docx
 
#  [2] With FOR LOOP:
#       [i] converts all files from type doc to docx
#      [ii] uses XML to read the word file and creates:
#           a dictionary(for searhing) and a list of all the lines(for model)

#  [3] Uses a sentence-transformers model 'all-MiniLM-L6-v1' that maps
#      sentences & paragraphs to a vector space and can be used for tasks like
#      clustering or semantic search.
#      If It won't work properly => change to  'all-MiniLM-L12-v2'

#  [4] Finds needed information+colors it and saves as new word [in 'Results']

#  [5] Prints the answer + saves as csv file and downloads it.


In [ ]:
#@title Helping stuff
# %-------------------------------%
# --------- Helping stuff ---------
# %-------------------------------%

# Making mount with the google drive
import google.colab
from google.colab import drive
drive.mount('/gdrive')

# AutoComplete
from google.colab import auth
auth.authenticate_user()

## Will display all of the packages installed in the virtual environment
# !python -m pip list

In [ ]:
#@title Import and install if needed
# %----------------------------------------------%
# --------- Import and install if needed ---------
# %-----------------------------------------------%

![ ! -f "pip_installed" ] && pip -q install --upgrade google-api-python-client

![ ! -f "pip_installed" ] && pip -q install --upgrade jedi==0.17.2

![ ! -f "pip_installed" ] && pip -q install --upgrade textract
import textract

![ ! -f "pip_installed" ] && pip -q install --upgrade python-docx
import docx
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

![ ! -f "pip_installed" ] && pip -q install --upgrade aspose.words
import aspose.words as aw

![ ! -f "pip_installed" ] && pip -q install --upgrade docx2python
import docx2python
from docx2python import docx2python

#### FOR NETWORK
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

![ ! -f "pip_installed" ] && pip -q install --upgrade umap-learn
# !pip install umap-learn
import umap

![ ! -f "pip_installed" ] && pip -q install --upgrade altair-viewer
# !pip install altair-viewer
import altair as alt
import altair_viewer


try:
    from xml.etree.cElementTree import XML
except ImportError:
    from xml.etree.ElementTree import XML

import zipfile
from zipfile import ZipFile

import csv
import io
import os
import re  # to find stuff in a string
import itertools
import requests    
import pandas as pd
import numpy as np

from sklearn.cluster import DBSCAN
from google.colab import files

# import win32com.client
# from docx.api import Document

In [ ]:
#@title Unzipping the CVs
# %-----------------------------------%
# --------- Unzipping the CVs ---------
# %-----------------------------------%

# specifying the zip file name
file_name = "../gdrive/My Drive/CVS.zip"

# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    # zip.printdir()

    # extract specific files
    zip.extract("CVS/CVs.zip", "../gdrive/My Drive")
    ZipFile("../gdrive/My Drive/CVS/CVs.zip", 'r').extractall("../gdrive/My Drive/CVS")
    zip.close()
os.remove("../gdrive/My Drive/CVS/CVs.zip")

In [ ]:
## remove
# import shutil
# shutil.rmtree("../gdrive/My Drive/CVS")
# shutil.rmtree("../gdrive/My Drive/CVS/CVs")

In [ ]:
# @title # Change_doc_to_docx ##
# %--------------------------------------------------------------------------%
# INPUT: dir_path, file_name, doc_path  OUTPUT: Saved as docx+returns new path
# %--------------------------------------------------------------------------%

def change_doc_to_docx(dir_path, file_name, doc_path):
    input = aw.Document(doc_path)
    output = aw.Document()
    output.remove_all_children()
    output.append_document(input, aw.ImportFormatMode.KEEP_SOURCE_FORMATTING)
    
    index_type = file_name.find('.') + 1
    doc_path = os.path.join(dir_path, file_name[:index_type]) + 'docx'
    output.save(doc_path)

    return doc_path

In [ ]:
#@title # Remove function#
# %-------------------------------------------------------------%
# Input: File path   Output: Dictionary of the text:  (line:text)
# %-------------------------------------------------------------%

def remove_t(paragraph, times, replace):
    for _ in range(times):
        paragraph = paragraph.replace(replace, " ")
    return paragraph

In [ ]:
# @title # YES (Version.1.2 - USING XML - GET DICTIOANRY + LIST) #
# %----------------------------------------------------------------%
# Input: File path   Output: List+Dictionary of the text (line:text)
# %----------------------------------------------------------------%

def get_docx_text_to_list_dict_XML(path):
    """Take the path of a docx file as argument, return the text in unicode."""
    WORD_NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
    PARA = WORD_NAMESPACE + 'p'
    TEXT = WORD_NAMESPACE + 't'
    # PAGE = WORD_NAMESPACE + 'lastRenderedPageBreak'

    document = zipfile.ZipFile(path)
    xml_content = document.read('word/document.xml').decode('utf-8')
    document.close()
    tree = XML(xml_content)
    
    lines = []
    docx_dict = {}

    for indx, paragraph in enumerate(tree.getiterator(PARA), start=1):
        texts = [n.text for n in paragraph.getiterator(TEXT) if n.text]        
        if texts:
            adding = ''.join(texts).lower()

            # cleaning
            if "\t" in adding:
                adding = remove_t(adding, adding.count("\t"), "\t")
            if "_" in adding:
                adding = remove_t(adding, adding.count("_"), "_")
            adding = adding.rstrip()

            if len(adding) != 0:
                lines.append(adding)
                docx_dict[indx] = adding
    return docx_dict, lines

In [ ]:
# @title NOPE (Version.2.2 - USING docx2python - GET DICTIOANRY + LIST)
# %----------------------------------------------------------------%
# Input: File path   Output: List+Dictionary of the text (line:text)
# %----------------------------------------------------------------%

def get_docx_text_to_list_dict_docx2python(path):    
    doc = docx2python(path)
    docx_dict = {}
    lines = []
    
    # 3 times itertools cause there are 3 levels
    tmp = list(itertools.chain(*doc.document))
    tmp = list(itertools.chain(*tmp))
    tmp = list(itertools.chain(*tmp))
    indx = 1

    for paragraph in tmp:
        # paragraph = paragraph.rstrip()
        if paragraph:
            # cleaning
            if "\t" in paragraph:
                paragraph = remove_t(paragraph, paragraph.count("\t"), "\t")
            if "_" in paragraph:
                paragraph = remove_t(paragraph, paragraph.count("_"), "_")
            paragraph = paragraph.rstrip()

            if len(paragraph) != 0:
                lines.append(paragraph.lower())                
                if "--" in paragraph[0:3]:
                    if indx not in docx_dict:
                        docx_dict[indx] = paragraph
                    else:                    
                        docx_dict[indx] = docx_dict[indx] + '  ' + paragraph
                else:            
                    indx += 1
                    docx_dict[indx] = paragraph
                    indx += 1                
                
    return docx_dict, lines

In [ ]:
#@title Example for myself - get_docx_text_to_dict
# %-------------------------------------------------------------------%
#---- Example for myself to find what is better XML or docx2python ----
# %-------------------------------------------------------------------%

d = "../gdrive/My Drive/CVS/CVs/"

# p = 'Data-Analyst-Resume-Example-MSWord-Download.docx'
# p = 'Bank-Teller-Resume-Sample-MSWord-Download.docx'
p = 'Graphic-Designer-Resume-Sample-MSWord-Download.docx'
# p = 'Electrician-Resume-Sample-MSWord-Download.docx'
# p = 'Retail-Manager-Resume-Sample-MSWord-Download.docx'

# return docx_dict, lines

# d1, l1 = get_docx_text_to_list_dict_XML((d+p))
d2, l2 = get_docx_text_to_list_dict_docx2python((d+p))

# d2

#########>>>>#########>>>>#########>>>>#########>>>>#########>>>>
# >>>>>>> get_docx_text_to_list_dict_XML is the best # <<<<<<<
#########>>>>#########>>>>#########>>>>#########>>>>#########>>>>

In [ ]:
# %-------------------------------------------------------------------%
#---- FOR LOOP that builds:   [1] docx_content = {}  [2] lines = [] ----
# %-------------------------------------------------------------------%

# LIST OF FILES
dir_path = "../gdrive/My Drive/CVS/CVs"
lines = []
docx_content = {}

# Iterate directory
for file_name in os.listdir(dir_path):    
    path_to_doc = os.path.join(dir_path, file_name)

    # check if current file is a doc
    if file_name.endswith(".doc"):
        path_to_doc = change_doc_to_docx(dir_path, file_name, path_to_doc)

    # check if current path is a file - then build dict+list:
    if os.path.isfile(path_to_doc):
        # tmp_dict, tmp_lines = get_docx_text_to_list_dict_XML(path_to_doc)
        tmp_dict, tmp_lines = get_docx_text_to_list_dict_docx2python(path_to_doc)

        lines += tmp_lines
        docx_content[file_name] = tmp_dict

In [ ]:
len(lines)
# lines

In [ ]:
#@title Example for myself - docx_content
# %--------------------------%
#----- Example for myself ----
# %---------------------------%

# p = 'School-Bus-Driver-Resume-Sample-MSWord-Download.docx'
# p = 'Sales-Associate-Resume-Sample-MSWord-Download (1).docx'
# p = 'Nanny-Resume-Sample-MSWord-Download.docx'
p = 'Bank-Teller-Resume-Sample-MSWord-Download.docx'
# p = 'combination-janitor-resume-sample-MSWord-download.docx'
# p = 'Electrician-Resume-Sample-MSWord-Download.docx'

a = docx_content[p].values()
# a

In [ ]:
# %---------------------------------------------------%
# ----- USING A MODEL named - 'all-MiniLM-L12-v2' -----
# ---------- sentence -> vector in R^n ----------
# %---------------------------------------------------%

# model = SentenceTransformer('all-MiniLM-L12-v2')
model = SentenceTransformer('all-MiniLM-L6-v1')
# embeddings_model = model.encode(lines)
## R^n -> R^2

In [ ]:
# %----------------------------%
# -----  CLASS to design   -----
# %----------------------------%

class color:
  PURPLE = '\033[95m'
  CYAN = '\033[96m'
  DARKCYAN = '\033[36m'
  BLUE = '\033[94m'
  GREEN = '\033[92m'
  YELLOW = '\033[93m'
  RED = '\033[91m'
  BOLD = '\033[1m'
  UNDERLINE = '\033[4m'
  END = '\033[0m'

In [ ]:
# %-----------------------------------------------------------%
# -----  importing WD_COLOR_INDEX and defing cos_angle   -----
# %-----------------------------------------------------------%

from docx.enum.text import WD_COLOR_INDEX
gdrive = '../gdrive/My Drive/CVS/'
try:
    os.mkdir(gdrive+'Results')
except:
    pass


def cos_angle(line,target):
    # sentence -> R^n
    line = model.encode(line)
    target = model.encode(target)
    return np.abs(np.inner(line,target) / np.linalg.norm(line) / np.linalg.norm(target))


list_we_want = ['name', 'email', 'experience', 'education']

In [ ]:
# %------------------------------------------------------------------------%
# --------------------------------  ANSWER  --------------------------------
# -----  For loop that finds - (name, email, education, experience)   -----
# %------------------------------------------------------------------------%

# with open(gdrive+'cv_details_all-MiniLM-L12-v2.csv', 'w', encoding='utf-8-sig') as file:
with open(gdrive+'cv_details_all-MiniLM-L6-v1.csv', 'w', encoding='utf-8-sig') as file:
    names_upper = [name.upper() for name in list_we_want]
    writer = csv.writer(file)
    writer.writerow(names_upper)

    for file_name, dict_chosen_file in docx_content.items():
        # create a document to save new colored data
        doc = docx.Document()
        
        d = {'name'                     : [],
            'email'                    : [],
            'experience'               : [],
            'education'                : [],
            'additional skills'        : [],
            'awards'                   : [],
            'licenses and certificates': [],
            'languages'                : [],
            'rest'                     : [],
        }

        # the last title
        saving = 'rest'

        for line_num, line_value in dict_chosen_file.items():
            threshold = 0.5
            line = line_value              

            if 'resume' in line:
                where = line.find('resume') - 1
                d['name'] = [line[:where]]                        
            
            elif '@' in line:
                match = re.findall(r'[\w.+-]+@[\w-]+\.[\w.-]+', line)
                d['email'] = [match[0]]
            
            elif cos_angle(line, 'experience') > threshold:
                saving = 'experience'
            
            # cos_angle('elementary teacher', 'teacher') = 0.8646255
            elif cos_angle(line, 'education') > threshold \
                and cos_angle(line, 'skills') < 0.8    \
                and cos_angle(line, 'teacher') < 0.8:
                  saving = 'education'

            elif cos_angle(line, 'additional skills') > threshold:
                saving = 'additional skills'
            
            elif cos_angle(line, 'awards') > threshold:
                saving = 'awards'

            elif cos_angle(line, 'licenses and certificates') > threshold:
                saving = 'licenses and certificates'
            
            elif cos_angle(line, 'languages') > threshold:
                saving = 'languages'
            
            if len(d['name']) == 0:            
                where2 = file_name.lower().find('resume') - 1
                tmp_name = file_name[:where2].lower()
                if '-' in tmp_name:
                    tmp_name = remove_t(tmp_name, tmp_name.count("-"), '-')
                if '_' in tmp_name:
                    tmp_name = remove_t(tmp_name, tmp_name.count("_"), '_')                
                d['name'] = [tmp_name]


            d[saving].append(line)

            # coloring the file:
            # [PINK = name, TURQUOISE = email, YELLOW = experience, BRIGHT_GREEN = education]
            if 'resume' in line:
                line1 = line[:where]
                line2 = line[where:]
                p = doc.add_paragraph()
                p.style.font.name = 'Arial'
                p.style.font.size=Pt(10)
                p.paragraph_format.space_after = 1

                p.add_run(line1).font.highlight_color = WD_COLOR_INDEX.PINK
                p.add_run(line2).font.highlight_color = WD_COLOR_INDEX.AUTO
                p.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER


            elif '@' in line:
                p = doc.add_paragraph()
                p.style.font.name = 'Arial'
                p.style.font.size=Pt(10)
                p.paragraph_format.space_after = 1
                chuncks = re.split(r'([\w.+-]+@[\w-]+\.[\w.-]+)', line)
                p.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
                for chunck in chuncks:
                    if re.fullmatch(r'[\w.+-]+@[\w-]+\.[\w.-]+', chunck):
                        p.add_run(chunck).font.highlight_color = WD_COLOR_INDEX.TURQUOISE
                    else:
                        p.add_run(chunck).font.highlight_color = WD_COLOR_INDEX.AUTO

            elif saving == 'experience':
                p = doc.add_paragraph()                
                p.paragraph_format.space_after = 1
                p.add_run(line).font.highlight_color = WD_COLOR_INDEX.YELLOW

            elif saving == 'education':
                p = doc.add_paragraph()                
                p.paragraph_format.space_after = 1
                p.add_run(line).font.highlight_color = WD_COLOR_INDEX.BRIGHT_GREEN

            else:
                p = doc.add_paragraph()                
                p.paragraph_format.space_after = 1
                p.add_run(line).font.highlight_color = WD_COLOR_INDEX.AUTO
        
        add_to_csv = []
        for key,value in d.items():
            if key in list_we_want:
                tmp_val = ' '.join(value)
                print(f"{color.GREEN + key + color.END} = {tmp_val}")
                add_to_csv.append(tmp_val)
        print('\n')
        writer.writerow(add_to_csv)
        doc.save(gdrive+'Results/'+file_name+'_modified.docx')

files.download(gdrive+'cv_details_all-MiniLM-L6-v1.csv')
# files.download(gdrive+'cv_details_all-MiniLM-L12-v2.csv')

In [ ]:
# files.download(gdrive+'cv_details_all-MiniLM-L6-v1.csv')